# Phase 1: Data Preparation

**Objective**: Complete the data preparation phase for Modbus Anomaly Detection project

**Step Overview**:
1. Step 1: PCAP File Parsing - Parse all PCAP files to extract Modbus packets
2. Step 2: Apply Labeling Policy - Label Attack/Normal based on IP and write operations
3. Step 3: 15-Second Window Feature Extraction - Calculate 44 features
4. **Step 3.5: Minimum Packet Count Filtering** - Filter windows with packet_count < 30 (ensure feature validity)
5. Step 4: Dataset Split - Split into Train/Validation/Test sets by PCAP files

**Estimated Duration**: Step 1 takes 4-6 hours, other steps take minutes to 1 hour each

---

## 0. Environment Setup and Configuration

In [23]:
#   Import dependencies
import os
import sys
import glob
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

#   Setup display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

#   Add src directory to path - use absolute path for correctness
PROJECT_ROOT = Path(r"c:\Users\Echo\Desktop\modbus-detection")
SRC_DIR = PROJECT_ROOT / 'src'

#   Ensure src directory is in Python path
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print(f"Project root: {PROJECT_ROOT}")
print(f"src directory: {SRC_DIR}")
print(f"src directory exists: {SRC_DIR.exists()}")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

项目根目录: c:\Users\Echo\Desktop\modbus-detection
src目录: c:\Users\Echo\Desktop\modbus-detection\src
src目录存在: True
当前时间: 2026-01-09 16:36:33


In [24]:
#   Import project modules
import config
from pcap_parser import (
    ModbusPacketParser, 
    scan_pcap_files, 
    parse_all_pcaps,
    save_packets_to_parquet,
    load_packets_from_parquet
)
from feature_extractor import (
    extract_window_features,
    get_feature_names,
    get_feature_groups,
    ATTACKER_IP,
    WRITE_FCS
)

#   Ensure directories exist
config.ensure_dirs()

print("✓ Modules imported successfully")

✓ 所有目录已创建
✓ 模块导入成功


In [25]:
#   Configuration parameters
PARAMS = {
    'time_window': 15,  #   Seconds, main experiment window size
    'random_seed': 42,
    'split_ratios': {
        'train': 0.70,
        'val': 0.15,
        'test': 0.15
    }
}

#   Output file paths
OUTPUT_PATHS = {
    'packets_raw': config.DATA_PROCESSED / 'packets_raw.parquet',
    'packets_labeled': config.DATA_PROCESSED / 'packets_labeled.parquet',
    'features_15s': config.DATA_PROCESSED / 'features_15s.parquet',
    'train': config.DATA_SPLITS / 'train.parquet',
    'val': config.DATA_SPLITS / 'val.parquet',
    'test': config.DATA_SPLITS / 'test.parquet',
    'split_info': config.DATA_SPLITS / 'split_info.json'
}

print("Configuration parameters:")
for k, v in PARAMS.items():
    print(f"  {k}: {v}")

print("\nOutput paths:")
for k, v in OUTPUT_PATHS.items():
    print(f"  {k}: {v}")

配置参数:
  time_window: 15
  random_seed: 42
  split_ratios: {'train': 0.7, 'val': 0.15, 'test': 0.15}

输出路径:
  packets_raw: c:\Users\Echo\Desktop\modbus-detection\data\processed\packets_raw.parquet
  packets_labeled: c:\Users\Echo\Desktop\modbus-detection\data\processed\packets_labeled.parquet
  features_15s: c:\Users\Echo\Desktop\modbus-detection\data\processed\features_15s.parquet
  train: c:\Users\Echo\Desktop\modbus-detection\data\splits\train.parquet
  val: c:\Users\Echo\Desktop\modbus-detection\data\splits\val.parquet
  test: c:\Users\Echo\Desktop\modbus-detection\data\splits\test.parquet
  split_info: c:\Users\Echo\Desktop\modbus-detection\data\splits\split_info.json


In [26]:
#   Check dataset paths
print("Dataset path check:")
print(f"  Dataset root: {config.DATASET_ROOT}")
print(f"  Exists: {os.path.exists(config.DATASET_ROOT)}")

print("\nPCAP directory check:")
pcap_dirs = {
    'Benign': config.BENIGN_PCAP_DIR,
    'External': config.EXTERNAL_PCAP_DIR,
    'SCADA': config.SCADA_PCAP_DIR,
    'IED': config.IED_PCAP_DIR
}

for name, path in pcap_dirs.items():
    exists = os.path.exists(path)
    file_count = len(glob.glob(os.path.join(path, '*.pcap'))) if exists else 0
    status = "✓" if exists else "✗"
    print(f"  {status} {name}: {file_count} PCAP files")
    if not exists:
        print(f"     Path: {path}")

数据集路径检查:
  数据集根目录: C:\Users\Echo\Desktop\Modbus Dataset\Modbus Dataset
  存在: True

PCAP目录检查:
  ✓ Benign: 19 个PCAP文件
  ✓ External: 1 个PCAP文件
  ✓ SCADA: 18 个PCAP文件
  ✓ IED: 6 个PCAP文件


---

## 1. Step 1: PCAP File Parsing

**Objective**: Parse all PCAP files to extract Modbus packet data

**Output**: `packets_raw.parquet`

**Estimated Duration**: 4-6 hours (can run overnight)

### 1.1 Scanning All PCAP Files

In [27]:
#   Scan PCAP files from each scenario
benign_files = scan_pcap_files(config.BENIGN_PCAP_DIR, 'benign')
external_files = scan_pcap_files(config.EXTERNAL_PCAP_DIR, 'external')
scada_files = scan_pcap_files(config.SCADA_PCAP_DIR, 'scada')
ied_files = scan_pcap_files(config.IED_PCAP_DIR, 'ied')

#   Merge all files
all_pcap_files = benign_files + external_files + scada_files + ied_files

print("\nPCAP file statistics:")
print(f"  Benign: {len(benign_files)}")
print(f"  External: {len(external_files)}")
print(f"  SCADA: {len(scada_files)}")
print(f"  IED: {len(ied_files)}")
print(f"  Total: {len(all_pcap_files)}")

2026-01-09 16:36:40,967 - INFO - 在 benign 目录找到 19 个PCAP文件
2026-01-09 16:36:40,969 - INFO - 在 external 目录找到 1 个PCAP文件
2026-01-09 16:36:40,970 - INFO - 在 scada 目录找到 18 个PCAP文件
2026-01-09 16:36:40,971 - INFO - 在 ied 目录找到 6 个PCAP文件



PCAP文件统计:
  Benign: 19
  External: 1
  SCADA: 18
  IED: 6
  总计: 44


In [28]:
#   View file list examples
print("Benign file examples:")
for f, s in benign_files[:3]:
    print(f"  {os.path.basename(f)}")

print("\nExternal file examples:")
for f, s in external_files[:5]:
    print(f"  {os.path.basename(f)}")

Benign文件示例:
  network-wide-normal-14.pcap
  network-wide-normal-15.pcap
  network-wide-normal-16.pcap

External文件示例:
  veth665f3cf-0.pcap


### 1.2 Parse PCAP Files

⚠️ **Note**: This step takes a long time (4-6 hours), recommend running overnight

In [18]:
#   Check if parsed results already exist
if OUTPUT_PATHS['packets_raw'].exists():
    print(f"✓ Found existing parsed results: {OUTPUT_PATHS['packets_raw']}")
    print("  To re-parse, delete this file and run again")
    
    #   Load existing data
    df_packets_raw = load_packets_from_parquet(str(OUTPUT_PATHS['packets_raw']))
    SKIP_PARSING = True
else:
    print("No existing parsed results found, will start parsing...")
    SKIP_PARSING = False

未发现已有解析结果，将开始解析...


In [19]:
#   Execute parsing (if needed)
if not SKIP_PARSING:
    print(f"Starting to parse {len(all_pcap_files)} PCAP files...")
    print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("Estimated time: 4-6 hours\n")
    
    #   Create parser
    parser = ModbusPacketParser(strict_mode=True)
    
    #   Batch parsing
    df_packets_raw = parse_all_pcaps(all_pcap_files, parser, show_progress=True)
    
    print(f"\nCompletion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Parsing result: {len(df_packets_raw):,} records")

开始解析 44 个PCAP文件...
开始时间: 2026-01-09 15:11:57
预计耗时: 4-6小时



解析PCAP文件:   0%|          | 0/44 [00:24<?, ?it/s]


: 

In [ ]:
#   Save raw data
if not SKIP_PARSING:
    save_packets_to_parquet(df_packets_raw, str(OUTPUT_PATHS['packets_raw']))
    print("✓ Raw data saved")

: 

In [ ]:
#   Emergency save - use CSV format (no pyarrow needed)
csv_path = str(OUTPUT_PATHS['packets_raw']).replace('.parquet', '.csv')
df_packets_raw.to_csv(csv_path, index=False)
print(f"✓ Data saved as CSV: {csv_path}")
print(f"  Records: {len(df_packets_raw):,}")

✓ 数据已保存为CSV: c:\Users\Echo\Desktop\modbus-detection\data\processed\packets_raw.csv
  记录数: 8,478,926


In [ ]:
#   Load already parsed data from CSV (skip 3 hours of parsing)
import pandas as pd

csv_path = config.DATA_PROCESSED / 'packets_raw.csv'
print(f"Loading data from CSV: {csv_path}")

df_packets_raw = pd.read_csv(csv_path)
df_packets_raw['timestamp'] = pd.to_datetime(df_packets_raw['timestamp'])

print(f"✓ Data loaded: {len(df_packets_raw):,} records")
print(f"\nScenario distribution:")
print(df_packets_raw['scenario'].value_counts())

#   Set flag to skip parsing
SKIP_PARSING = True

从CSV加载数据: c:\Users\Echo\Desktop\modbus-detection\data\processed\packets_raw.csv
✓ 数据已加载: 8,478,926 条记录

场景分布:
scenario
benign      3953147
scada       3344142
ied         1116026
external      65611
Name: count, dtype: int64


### 1.3 Data Quality Check

In [ ]:
#   Basic info
print("Raw data overview:")
print(f"  Total records: {len(df_packets_raw):,}")
print(f"  Columns: {len(df_packets_raw.columns)}")
print(f"  Column names: {list(df_packets_raw.columns)}")

print("\nData types:")
print(df_packets_raw.dtypes)

In [ ]:
#   Scenario distribution
print("Scenario distribution:")
scenario_counts = df_packets_raw['scenario'].value_counts()
for scenario, count in scenario_counts.items():
    pct = count / len(df_packets_raw) * 100
    print(f"  {scenario}: {count:,} ({pct:.1f}%)")

In [ ]:
#   Function code distribution
print("Function code distribution:")
fc_counts = df_packets_raw['function_code'].value_counts().sort_index()
for fc, count in fc_counts.items():
    pct = count / len(df_packets_raw) * 100
    fc_type = 'Write' if fc in WRITE_FCS else 'Read'
    print(f"  FC {fc:2d} ({fc_type}): {count:,} ({pct:.1f}%)")

In [ ]:
#   IP address distribution
print("Source IP distribution:")
src_ip_counts = df_packets_raw['src_ip'].value_counts()
for ip, count in src_ip_counts.head(10).items():
    pct = count / len(df_packets_raw) * 100
    print(f"  {ip}: {count:,} ({pct:.1f}%)")

print("\nDestination IP distribution:")
dst_ip_counts = df_packets_raw['dst_ip'].value_counts()
for ip, count in dst_ip_counts.head(10).items():
    pct = count / len(df_packets_raw) * 100
    print(f"  {ip}: {count:,} ({pct:.1f}%)")

In [ ]:
#   Check missing values
print("Missing value statistics:")
missing = df_packets_raw.isnull().sum()
for col, count in missing.items():
    if count > 0:
        pct = count / len(df_packets_raw) * 100
        print(f"  {col}: {count:,} ({pct:.1f}%)")

if missing.sum() == 0:
    print("  No missing values (except start_address and quantity, which is expected)")

In [ ]:
#   Data sample
print("Data sample (first 5 rows):")
df_packets_raw.head()

---

## 2. Step 2: Apply Labeling Policy

**Objective**: Label each packet according to design document labeling policy

**Labeling Rules**:
- Benign: All packets → Normal
- External: src_ip == 185.175.0.7 → Attack, others → Normal
- Compromised (SCADA/IED): Write operations (FC 5/6/15/16) → marked, final label determined at window level

**Output**: `packets_labeled.parquet`

In [ ]:
#   Copy data
df_labeled = df_packets_raw.copy()

#   Initialize label columns
df_labeled['is_write_operation'] = df_labeled['function_code'].isin(WRITE_FCS)
df_labeled['is_external_attacker'] = df_labeled['src_ip'] == ATTACKER_IP
df_labeled['packet_label'] = 'unknown'

print("✓ Label columns initialized")

✓ 标签列已初始化


In [ ]:
#   Apply Benign scenario labels
benign_mask = df_labeled['scenario'] == 'benign'
df_labeled.loc[benign_mask, 'packet_label'] = 'normal'

print(f"Benign scenario: {benign_mask.sum():,} packets → Normal")

Benign场景: 3,953,147 包 → Normal


In [ ]:
#   Apply External scenario labels (IP filtering)
external_mask = df_labeled['scenario'] == 'external'
external_attack_mask = external_mask & df_labeled['is_external_attacker']
external_normal_mask = external_mask & ~df_labeled['is_external_attacker']

df_labeled.loc[external_attack_mask, 'packet_label'] = 'attack'
df_labeled.loc[external_normal_mask, 'packet_label'] = 'normal'

print(f"External scenario:")
print(f"  Attack (src_ip={ATTACKER_IP}): {external_attack_mask.sum():,} packets")
print(f"  Normal (other IPs): {external_normal_mask.sum():,} packets")

External场景:
  Attack (src_ip=185.175.0.7): 65,611 包
  Normal (其他IP): 0 包


In [ ]:
#   Apply Compromised scenario labels (write operation marking)
#   Note: Mark as 'attack_env' first, final label determined at window level
compromised_mask = df_labeled['scenario'].isin(['scada', 'ied'])
df_labeled.loc[compromised_mask, 'packet_label'] = 'attack_env'

print(f"Compromised scenario (SCADA + IED): {compromised_mask.sum():,} packets → attack_env")
print(f"  Write operation packets: {(compromised_mask & df_labeled['is_write_operation']).sum():,}")

Compromised场景 (SCADA + IED): 4,460,168 包 → attack_env
  其中写操作包: 521,693


In [ ]:
#   Label distribution statistics
print("Packet-level label distribution:")
label_counts = df_labeled['packet_label'].value_counts()
for label, count in label_counts.items():
    pct = count / len(df_labeled) * 100
    print(f"  {label}: {count:,} ({pct:.1f}%)")

包级别标签分布:
  attack_env: 4,460,168 (52.6%)
  normal: 3,953,147 (46.6%)
  attack: 65,611 (0.8%)


In [ ]:
#   Save labeled data
save_packets_to_parquet(df_labeled, str(OUTPUT_PATHS['packets_labeled']))
print("✓ Labeled data saved")

2026-01-09 15:11:49,508 - INFO - 数据已保存: c:\Users\Echo\Desktop\modbus-detection\data\processed\packets_labeled.parquet
2026-01-09 15:11:49,510 - INFO -   - 记录数: 8,478,926
2026-01-09 15:11:49,511 - INFO -   - 文件大小: 103.38 MB


✓ 标记后数据已保存


---

## 3. Step 3: 15-Second Window Feature Extraction

**Objective**: Aggregate packet-level data into 15-second windows, calculate 44 features

**Window Labeling Rules**:
- Benign windows: Normal
- External windows: Has attacker IP → Attack, otherwise → Normal
- Compromised windows: Has write operations → Attack, otherwise → Normal

**Output**: `features_15s.parquet`

In [20]:
#   Feature info
feature_names = get_feature_names()
feature_groups = get_feature_groups()

print(f"Total features: {len(feature_names)}")
print("\nFeature groups:")
for group, features in feature_groups.items():
    print(f"  {group}: {len(features)}")

特征总数: 44

特征分组:
  protocol: 13个
  temporal: 9个
  dcs_device_role: 6个
  dcs_topology: 5个
  dcs_operation: 6个
  dcs_anomaly: 5个


In [21]:
#   Ensure timestamp is datetime type
df_labeled['timestamp'] = pd.to_datetime(df_labeled['timestamp'])

#   Group by scenario and file
grouped = df_labeled.groupby(['scenario', 'pcap_file'])
print(f"Number of groups: {len(grouped)}")

分组数: 43


In [22]:
def create_windows(df_file, window_size_sec=15):
    """
    Split a single file's data into time windows
    
    Args:
        df_file: Data from a single PCAP file
        window_size_sec: Window size (seconds)
    
    Returns:
        List of windows, each window is a DataFrame
    """
    if len(df_file) == 0:
        return []
    
    #   Sort by time
    df_sorted = df_file.sort_values('timestamp')
    
    #   Get time range
    start_time = df_sorted['timestamp'].min()
    end_time = df_sorted['timestamp'].max()
    
    #   Create windows
    windows = []
    current_start = start_time
    window_delta = pd.Timedelta(seconds=window_size_sec)
    
    while current_start < end_time:
        current_end = current_start + window_delta
        
        #   Filter packets within window
        mask = (df_sorted['timestamp'] >= current_start) & (df_sorted['timestamp'] < current_end)
        window_df = df_sorted[mask]
        
        if len(window_df) > 0:
            windows.append({
                'window_start': current_start,
                'window_end': current_end,
                'data': window_df
            })
        
        current_start = current_end
    
    return windows

In [23]:
def determine_window_label(window_df, scenario):
    """
    Determine window label
    
    Args:
        window_df: Packet data within window
        scenario: Scenario identifier
    
    Returns:
        'Attack' or 'Normal'
    """
    if scenario == 'benign':
        return 'Normal'
    
    elif scenario == 'external':
        #   External: Has attacker IP → Attack
        if window_df['is_external_attacker'].any():
            return 'Attack'
        else:
            return 'Normal'
    
    elif scenario in ['scada', 'ied']:
        #   Compromised: Has write operations → Attack
        if window_df['is_write_operation'].any():
            return 'Attack'
        else:
            return 'Normal'
    
    else:
        return 'Unknown'

In [24]:
#   Extract features for all windows
print(f"Starting 15-second window feature extraction...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

all_window_features = []
window_size = PARAMS['time_window']

for (scenario, pcap_file), df_file in tqdm(grouped, desc="Processing files"):
    #   Split into windows
    windows = create_windows(df_file, window_size_sec=window_size)
    
    for window in windows:
        window_df = window['data']
        
        #   Extract features
        features = extract_window_features(window_df, window_size)
        
        #   Add metadata
        features['scenario'] = scenario
        features['pcap_file'] = pcap_file
        features['window_start'] = window['window_start']
        features['window_end'] = window['window_end']
        features['packet_count_raw'] = len(window_df)  #   Raw packet count (should match packet_count in features)
        
        #   Determine window label
        features['label'] = determine_window_label(window_df, scenario)
        
        all_window_features.append(features)

print(f"\nCompletion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total windows: {len(all_window_features)}")

开始提取15秒窗口特征...
开始时间: 2026-01-09 15:13:27


处理文件:   0%|          | 0/43 [00:00<?, ?it/s]


完成时间: 2026-01-09 15:50:55
总窗口数: 121934


In [25]:
#   Convert to DataFrame
df_features = pd.DataFrame(all_window_features)

print(f"Feature dataset shape: {df_features.shape}")
print(f"Columns: {len(df_features.columns)} (44 features + 6 metadata)")

特征数据集形状: (121934, 50)
列数: 50 (44个特征 + 6个元数据)


In [26]:
#   Save feature data
save_packets_to_parquet(df_features, str(OUTPUT_PATHS['features_15s']))
print("✓ Feature data saved")

2026-01-09 15:52:15,295 - INFO - 数据已保存: c:\Users\Echo\Desktop\modbus-detection\data\processed\features_15s.parquet
2026-01-09 15:52:15,296 - INFO -   - 记录数: 121,934
2026-01-09 15:52:15,297 - INFO -   - 文件大小: 7.55 MB


✓ 特征数据已保存


### 3.1 Feature Dataset Statistics

In [27]:
#   Label distribution
print("Window-level label distribution:")
label_counts = df_features['label'].value_counts()
for label, count in label_counts.items():
    pct = count / len(df_features) * 100
    print(f"  {label}: {count:,} ({pct:.1f}%)")

print(f"\nClass ratio (Normal:Attack): {label_counts.get('Normal', 0)}:{label_counts.get('Attack', 0)}")
if label_counts.get('Attack', 0) > 0:
    ratio = label_counts.get('Normal', 0) / label_counts.get('Attack', 0)
    print(f"  Ratio: {ratio:.1f}:1")

窗口级标签分布:
  Normal: 116,587 (95.6%)
  Attack: 5,347 (4.4%)

类别比例 (Normal:Attack): 116587:5347
  比例: 21.8:1


In [28]:
#   Scenario distribution
print("Scenario distribution:")
scenario_counts = df_features['scenario'].value_counts()
for scenario, count in scenario_counts.items():
    pct = count / len(df_features) * 100
    print(f"  {scenario}: {count:,} ({pct:.1f}%)")

场景分布:
  benign: 44,904 (36.8%)
  scada: 39,137 (32.1%)
  ied: 37,846 (31.0%)
  external: 47 (0.0%)


In [29]:
#   Scenario × Label cross table
print("Scenario × Label cross table:")
cross_tab = pd.crosstab(df_features['scenario'], df_features['label'])
print(cross_tab)

场景 × 标签交叉表:
label     Attack  Normal
scenario                
benign         0   44904
external      47       0
ied         1487   36359
scada       3813   35324


In [30]:
#   Feature statistical description
print("Feature statistical description:")
df_features[feature_names].describe()

特征统计描述:


,fc_distribution_entropy,fc_diversity,fc_read_ratio,fc_write_ratio,txid_mean,txid_std,txid_unique_ratio,unit_id_diversity,packet_size_mean,packet_size_std,address_range,value_mean,value_std,packet_count,packet_rate,interval_mean,interval_std,interval_min,interval_max,burst_count,burst_intensity,session_count,scada_src_ratio,scada_dst_ratio,ied_src_ratio,ied_dst_ratio,device_role_entropy,is_typical_scada_to_ied,src_ip_count,dst_ip_count,comm_pair_count,dominant_pair_ratio,multi_target_ratio,consecutive_write_max,consecutive_write_mean,write_burst_count,read_write_alternation,operation_sequence_entropy,write_without_read_ratio,external_ip_present,external_ip_packet_ratio,unknown_ip_count,abnormal_fc_ratio,address_range_exceeded
count,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.0,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.0,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.00000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.000000,121934.0,121934.0,121934.000000
mean,1.931346,4.085308,0.993173,0.006827,31337.756516,9668.896077,0.500986,1.0,11.224371,0.921551,42.766513,9.931757,23.360324,69.537012,4.635801,0.221806,0.831613,0.000233,3.997519,5.315810,11.913992,4.492250,0.504437,0.495178,0.495178,0.504822,0.0,0.999615,3.187995,3.303820,4.492250,0.297255,0.18516,2.710097,1.321608,0.021741,0.003931,0.022482,0.000074,0.000385,0.000385,0.0,0.0,0.000044
std,0.105305,0.318107,0.053328,0.053328,14207.371705,7884.634178,0.034179,0.0,0.287931,1.173410,348.148140,413.147931,488.688594,129.013854,8.600924,0.104192,0.317174,0.000125,0.968254,2.230681,45.819743,1.821498,0.027270,0.027205,0.027205,0.027205,0.0,0.019629,0.930113,0.928247,1.821498,0.154616,0.13683,92.511626,75.384169,0.448722,0.016487,0.087763,0.008591,0.019629,0.019629,0.0,0.0,0.001698
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.002828,1.0,10.782609,0.000000,0.000000,0.000000,0.000000,1.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,1.000000,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
25%,1.921928,4.000000,1.000000,0.000000,21300.515625,4.320494,0.500000,1.0,11.200000,0.871780,30.000000,1.000000,0.000000,30.000000,2.000000,0.133548,0.574454,0.000178,3.177913,3.000000,9.000000,2.000000,0.500000,0.500000,0.500000,0.500000,0.0,1.000000,2.000000,2.000000,2.000000,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
50%,1.921928,4.000000,1.000000,0.000000,31322.827381,11628.714604,0.500000,1.0,11.200000,0.871780,30.000000,1.000000,0.000000,80.000000,5.333333,0.161496,0.673596,0.000235,4.117591,6.000000,9.000000,6.000000,0.500000,0.500000,0.500000,0.500000,0.0,1.000000,4.000000,4.000000,6.000000,0.187500,0.25000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
75%,1.921928,4.000000,1.000000,0.000000,40797.511905,16306.956968,0.500000,1.0,11.200000,0.871780,30.000000,1.000000,0.000000,90.000000,6.000000,0.354726,1.265617,0.000290,5.006182,6.000000,14.000000,6.000000,0.500000,0.500000,0.500000,0.500000,0.0,1.000000,4.000000,4.000000,6.000000,0.500000,0.25000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
max,2.418296,6.000000,1.000000,1.000000,65525.000000,32691.359547,1.000000,1.0,26.666667,60.472216,10282.000000,65535.000000,22609.628676,12639.000000,842.600000,2.006494,3.765320,0.014887,11.823034,24.000000,8800.000000,6.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,4.000000,4.000000,6.000000,1.000000,1.00000,1263

In [31]:
#   Data sample
print("Data sample (first 5 rows):")
df_features.head()

数据样例（前5行）:


,fc_distribution_entropy,fc_diversity,fc_read_ratio,fc_write_ratio,txid_mean,txid_std,txid_unique_ratio,unit_id_diversity,packet_size_mean,packet_size_std,address_range,value_mean,value_std,packet_count,packet_rate,interval_mean,interval_std,interval_min,interval_max,burst_count,burst_intensity,session_count,scada_src_ratio,scada_dst_ratio,ied_src_ratio,ied_dst_ratio,device_role_entropy,is_typical_scada_to_ied,src_ip_count,dst_ip_count,comm_pair_count,dominant_pair_ratio,multi_target_ratio,consecutive_write_max,consecutive_write_mean,write_burst_count,read_write_alternation,operation_sequence_entropy,write_without_read_ratio,external_ip_present,external_ip_packet_ratio,unknown_ip_count,abnormal_fc_ratio,address_range_exceeded,scenario,pcap_file,window_start,window_end,packet_count_raw,label
0,1.907589,4,1.0,0.0,16414.205882,271.537212,0.5,1,11.191176,0.878792,30.0,1.0,0.0,68,4.533333,0.223642,0.981275,0.000371,4.755910,4,16.000000,6,0.5,0.5,0.5,0.5,0.0,1.0,4,4,6,0.191176,0.25,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,benign,network-wide-normal-14.pcap,2023-01-24 04:52:05.777466,2023-01-24 04:52:20.777466,68,Normal
1,1.911311,4,1.0,0.0,16454.659091,282.421746,0.5,1,11.193182,0.877211,30.0,1.0,0.0,88,5.866667,0.120688,0.711788,0.000414,4.760056,3,28.333333,6,0.5,0.5,0.5,0.5,0.0,1.0,4,4,6,0.170455,0.25,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,benign,network-wide-normal-14.pcap,2023-01-24 04:52:20.777466,2023-01-24 04:52:35.777466,88,Normal
2,1.921928,4,1.0,0.0,16464.666667,281.222727,0.5,1,11.200000,0.871780,30.0,1.0,0.0,90,6.000000,0.118019,0.706653,0.000300,4.777379,6,14.000000,6,0.5,0.5,0.5,0.5,0.0,1.0,4,4,6,0.166667,0.25,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,benign,network-wide-normal-14.pcap,2023-01-24 04:52:35.777466,2023-01-24 04:52:50.777466,90,Normal
3,1.921928,4,1.0,0.0,16479.666667,281.222727,0.5,1,11.200000,0.871780,30.0,1.0,0.0,90,6.000000,0.118000,0.707162,0.000148,4.781165,6,14.000000,6,0.5,0.5,0.5,0.5,0.0,1.0,4,4,6,0.166667,0.25,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,benign,network-wide-normal-14.pcap,2023-01-24 04:52:50.777466,2023-01-24 04:53:05.777466,90,Normal
4,1.921928,4,1.0,0.0,16494.666667,281.222727,0.5,1,11.200000,0.871780,30.0,1.0,0.0,90,6.000000,0.117983,0.707165,0.000412,4.780578,6,14.000000,6,0.5,0.5,0.5,0.5,0.0,1.0,4,4,6,0.166667,0.25,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,benign,network-wide-normal-14.pcap,2023-01-24 04:53:05.777466,2023-01-24 04:53:20.777466,90,Normal


---

## 3.5 Step 3.5: Minimum Packet Count Filtering

**Objective**: Filter windows with too few packets to ensure statistical feature validity

**Rationale**: 
- Industrial control traffic is sparse, some windows have very few packets (<30 packets)
- 30 is the classic threshold for "large samples" in statistics
- With too few packets, statistical features like entropy and std dev are unstable

**Threshold**: `packet_count >= 30`

**Expected Result**: Retain about 60% of windows (~73,000 samples)

In [29]:
#   Load saved feature data
df_features = pd.read_parquet(str(OUTPUT_PATHS['features_15s']))
print(f"Feature data loaded: {len(df_features):,} windows")

已加载特征数据: 121,934 个窗口


In [30]:
#   Minimum packet count threshold configuration
MIN_PACKETS_THRESHOLD = 30

#   Pre-filtering statistics
print("=" * 60)
print("Step 3.5: Minimum Packet Count Filtering")
print("=" * 60)
print(f"\nWindows before filtering: {len(df_features):,}")
print(f"Minimum packet threshold: >= {MIN_PACKETS_THRESHOLD}")

#   View packet count distribution
print(f"\nWindow packet count distribution:")
print(f"  Min: {df_features['packet_count'].min()}")
print(f"  Max: {df_features['packet_count'].max()}")
print(f"  Mean: {df_features['packet_count'].mean():.1f}")
print(f"  Median: {df_features['packet_count'].median():.1f}")

#   Apply filtering
df_features_filtered = df_features[df_features['packet_count'] >= MIN_PACKETS_THRESHOLD].copy()

#   Post-filtering statistics
filtered_count = len(df_features_filtered)
original_count = len(df_features)
removed_count = original_count - filtered_count
retention_rate = filtered_count / original_count * 100

print(f"\nFiltering results:")
print(f"  Before: {original_count:,} windows")
print(f"  After: {filtered_count:,} windows")
print(f"  Removed: {removed_count:,} windows")
print(f"  Retention rate: {retention_rate:.1f}%")

Step 3.5: 最小包数过滤

过滤前窗口数: 121,934
最小包数阈值: >= 30

窗口包数分布:
  最小: 1
  最大: 12639
  平均: 69.5
  中位数: 80.0

过滤结果:
  过滤前: 121,934 个窗口
  过滤后: 118,209 个窗口
  移除: 3,725 个窗口
  保留率: 96.9%


In [31]:
#   Analyze filtering effect by scenario
print("\nFiltering effect by scenario:")
print("-" * 60)

for scenario in df_features['scenario'].unique():
    before = len(df_features[df_features['scenario'] == scenario])
    after = len(df_features_filtered[df_features_filtered['scenario'] == scenario])
    rate = after / before * 100 if before > 0 else 0
    avg_pkt = df_features[df_features['scenario'] == scenario]['packet_count'].mean()
    print(f"  {scenario}:")
    print(f"    Before: {before:,} → After: {after:,} (Retained {rate:.1f}%)")
    print(f"    Avg packets/window: {avg_pkt:.1f}")


按场景的过滤效果:
------------------------------------------------------------
  benign:
    过滤前: 44,904 → 过滤后: 44,821 (保留 99.8%)
    平均包数/窗口: 88.0
  external:
    过滤前: 47 → 过滤后: 7 (保留 14.9%)
    平均包数/窗口: 1396.0
  ied:
    过滤前: 37,846 → 过滤后: 34,582 (保留 91.4%)
    平均包数/窗口: 29.5
  scada:
    过滤前: 39,137 → 过滤后: 38,799 (保留 99.1%)
    平均包数/窗口: 85.4


In [32]:
#   Post-filtering label distribution
print("\nLabel distribution after filtering:")
print("-" * 60)

label_before = df_features['label'].value_counts()
label_after = df_features_filtered['label'].value_counts()

print("\n  Before filtering:")
for label, count in label_before.items():
    pct = count / len(df_features) * 100
    print(f"    {label}: {count:,} ({pct:.1f}%)")

print("\n  After filtering:")
for label, count in label_after.items():
    pct = count / len(df_features_filtered) * 100
    print(f"    {label}: {count:,} ({pct:.1f}%)")

#   Calculate class ratio
if 'Attack' in label_after.index and 'Normal' in label_after.index:
    ratio = label_after['Normal'] / label_after['Attack']
    print(f"\n  Normal:Attack ratio = {ratio:.1f}:1")


过滤后标签分布:
------------------------------------------------------------

  过滤前:
    Normal: 116,587 (95.6%)
    Attack: 5,347 (4.4%)

  过滤后:
    Normal: 112,991 (95.6%)
    Attack: 5,218 (4.4%)

  Normal:Attack 比例 = 21.7:1


In [33]:
#   Save filtered feature data
filtered_output_path = OUTPUT_PATHS['features_15s'].parent / 'features_15s_filtered.parquet'
save_packets_to_parquet(df_features_filtered, str(filtered_output_path))
print(f"\n✓ Filtered feature data saved: {filtered_output_path}")

#   Update df_features to filtered data (for Step 4)
df_features = df_features_filtered
print(f"✓ df_features updated to filtered data ({len(df_features):,} windows)")

2026-01-09 16:37:29,235 - INFO - 数据已保存: c:\Users\Echo\Desktop\modbus-detection\data\processed\features_15s_filtered.parquet
2026-01-09 16:37:29,236 - INFO -   - 记录数: 118,209
2026-01-09 16:37:29,237 - INFO -   - 文件大小: 7.31 MB



✓ 过滤后特征数据已保存: c:\Users\Echo\Desktop\modbus-detection\data\processed\features_15s_filtered.parquet
✓ df_features 已更新为过滤后数据 (118,209 个窗口)


---

## 4. Step 4: Dataset Split

**Objective**: Split into Train/Validation/Test sets by PCAP files (70/15/15)

**Principle**: Windows from the same file stay in the same set to prevent data leakage

**Output**: `train.parquet`, `val.parquet`, `test.parquet`

In [34]:
from sklearn.model_selection import train_test_split
import json

#   Get unique PCAP file list
unique_files = df_features['pcap_file'].unique()
print(f"Unique PCAP files: {len(unique_files)}")

唯一PCAP文件数: 43


In [35]:
#   Determine primary label for each file (for stratified split)
file_labels = df_features.groupby('pcap_file')['label'].apply(
    lambda x: 'Attack' if 'Attack' in x.values else 'Normal'
).reset_index()
file_labels.columns = ['pcap_file', 'file_label']

print("File-level label distribution:")
file_label_counts = file_labels['file_label'].value_counts()
print(file_label_counts)

文件级标签分布:
file_label
Attack    24
Normal    19
Name: count, dtype: int64


In [36]:
#   Stratified split
np.random.seed(PARAMS['random_seed'])

#   First split out test set (15%)
train_val_files, test_files = train_test_split(
    file_labels['pcap_file'].values,
    test_size=PARAMS['split_ratios']['test'],
    stratify=file_labels['file_label'].values,
    random_state=PARAMS['random_seed']
)

#   Then split validation set from remainder (15% of original = 15/85 of train_val)
val_ratio_adjusted = PARAMS['split_ratios']['val'] / (1 - PARAMS['split_ratios']['test'])

#   Get labels for train_val files
train_val_labels = file_labels[file_labels['pcap_file'].isin(train_val_files)]['file_label'].values

train_files, val_files = train_test_split(
    train_val_files,
    test_size=val_ratio_adjusted,
    stratify=train_val_labels,
    random_state=PARAMS['random_seed']
)

print(f"File split:")
print(f"  Train: {len(train_files)} files")
print(f"  Validation: {len(val_files)} files")
print(f"  Test: {len(test_files)} files")

文件划分:
  训练集: 29 个文件
  验证集: 7 个文件
  测试集: 7 个文件


In [37]:
#   Split data by files
df_train = df_features[df_features['pcap_file'].isin(train_files)].copy()
df_val = df_features[df_features['pcap_file'].isin(val_files)].copy()
df_test = df_features[df_features['pcap_file'].isin(test_files)].copy()

print(f"\nWindow split:")
print(f"  Train: {len(df_train):,} windows ({len(df_train)/len(df_features)*100:.1f}%)")
print(f"  Validation: {len(df_val):,} windows ({len(df_val)/len(df_features)*100:.1f}%)")
print(f"  Test: {len(df_test):,} windows ({len(df_test)/len(df_features)*100:.1f}%)")


窗口划分:
  训练集: 83,078 个窗口 (70.3%)
  验证集: 15,371 个窗口 (13.0%)
  测试集: 19,760 个窗口 (16.7%)


In [38]:
#   Check label distribution for each set
print("Label distribution per set:")
for name, df in [('Train', df_train), ('Validation', df_val), ('Test', df_test)]:
    label_dist = df['label'].value_counts()
    normal = label_dist.get('Normal', 0)
    attack = label_dist.get('Attack', 0)
    total = len(df)
    print(f"  {name}: Normal={normal} ({normal/total*100:.1f}%), Attack={attack} ({attack/total*100:.1f}%)")

各集合标签分布:
  训练集: Normal=79732 (96.0%), Attack=3346 (4.0%)
  验证集: Normal=14611 (95.1%), Attack=760 (4.9%)
  测试集: Normal=18648 (94.4%), Attack=1112 (5.6%)


In [39]:
#   Save split datasets
df_train.to_parquet(OUTPUT_PATHS['train'], index=False)
df_val.to_parquet(OUTPUT_PATHS['val'], index=False)
df_test.to_parquet(OUTPUT_PATHS['test'], index=False)

print("✓ Train/validation/test sets saved")

✓ 训练/验证/测试集已保存


In [42]:
from feature_extractor import get_feature_names
feature_names = get_feature_names()
#   Save split metadata
split_info = {
    'created_at': datetime.now().isoformat(),
    'params': PARAMS,
    'total_windows': len(df_features),
    'total_files': len(unique_files),
    'train': {
        'files': len(train_files),
        'windows': len(df_train),
        'normal': int(df_train['label'].value_counts().get('Normal', 0)),
        'attack': int(df_train['label'].value_counts().get('Attack', 0))
    },
    'val': {
        'files': len(val_files),
        'windows': len(df_val),
        'normal': int(df_val['label'].value_counts().get('Normal', 0)),
        'attack': int(df_val['label'].value_counts().get('Attack', 0))
    },
    'test': {
        'files': len(test_files),
        'windows': len(df_test),
        'normal': int(df_test['label'].value_counts().get('Normal', 0)),
        'attack': int(df_test['label'].value_counts().get('Attack', 0))
    },
    'feature_count': len(feature_names),
    'feature_names': feature_names
}

with open(OUTPUT_PATHS['split_info'], 'w', encoding='utf-8') as f:
    json.dump(split_info, f, indent=2, ensure_ascii=False)

print("✓ Split metadata saved")

✓ 划分元信息已保存


---

## 5. Summary and Next Steps

In [43]:
#   Phase 1 completion checklist
print("=" * 60)
print("Phase 1: Data Preparation - Completion Checklist")
print("=" * 60)

checklist = [
    ('packets_raw.parquet', OUTPUT_PATHS['packets_raw']),
    ('packets_labeled.parquet', OUTPUT_PATHS['packets_labeled']),
    ('features_15s.parquet', OUTPUT_PATHS['features_15s']),
    ('train.parquet', OUTPUT_PATHS['train']),
    ('val.parquet', OUTPUT_PATHS['val']),
    ('test.parquet', OUTPUT_PATHS['test']),
    ('split_info.json', OUTPUT_PATHS['split_info']),
]

all_exist = True
for name, path in checklist:
    exists = path.exists()
    status = "✓" if exists else "✗"
    size = f"{path.stat().st_size / (1024*1024):.2f} MB" if exists else "N/A"
    print(f"  {status} {name}: {size}")
    if not exists:
        all_exist = False

print("\n" + "=" * 60)
if all_exist:
    print("✓ Phase 1 data preparation complete!")
else:
    print("✗ Some files missing, please check")

Phase 1: 数据准备 - 完成检查清单
  ✗ packets_raw.parquet: N/A
  ✓ packets_labeled.parquet: 103.38 MB
  ✓ features_15s.parquet: 7.55 MB
  ✓ train.parquet: 5.15 MB
  ✓ val.parquet: 1.03 MB
  ✓ test.parquet: 1.26 MB
  ✓ split_info.json: 0.00 MB

✗ 部分文件缺失，请检查


In [44]:
#   Dataset statistics summary
print("\nDataset statistics summary:")
print(f"  Total windows: {len(df_features):,}")
print(f"  Features: {len(feature_names)}")
print(f"  Sample/feature ratio: {len(df_features)/len(feature_names):.1f}:1")
print(f"\nLabel distribution:")
for label, count in df_features['label'].value_counts().items():
    print(f"  {label}: {count:,} ({count/len(df_features)*100:.1f}%)")


数据集统计摘要:
  总窗口数: 118,209
  特征数: 44
  样本/特征比: 2686.6:1

标签分布:
  Normal: 112,991 (95.6%)
  Attack: 5,218 (4.4%)


In [ ]:
#   Phase 2 preparation items
print("\n" + "=" * 60)
print("Phase 2 Preparation:")
print("=" * 60)
print("""
1. Train RF main model
   - Train Random Forest on training set
   - Tune hyperparameters on validation set
   - Evaluate final performance on test set

2. 5-fold cross-validation
   - Report mean ± std dev
   - Ensure result stability

3. Basic performance evaluation
   - Accuracy, Precision, Recall, F1-Score
   - Confusion matrix
   - AUC-ROC curve
""")


Phase 2 准备事项:

1. 训练RF主模型
   - 使用训练集训练Random Forest
   - 在验证集上调优超参数
   - 在测试集上评估最终性能

2. 5-fold交叉验证
   - 报告均值±标准差
   - 确保结果稳定性

3. 基础性能评估
   - Accuracy, Precision, Recall, F1-Score
   - 混淆矩阵
   - AUC-ROC曲线



In [22]:
print(f"\nCompletion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


完成时间: 2026-01-09 16:18:56
